In [32]:
import fnv
import fnv.reduce
import fnv.file
import numpy as np
import seaborn as sns
import datetime as dt
import time
import pandas as pd
from tkinter import filedialog
from tkinter import *
import os
import matplotlib.pyplot as plt
import glob
import h5py
import gc
import shlex
import pipes
from subprocess import check_call




In [33]:
np.__version__

'1.19.2'

In [34]:
import torch

In [35]:
import torchvision

ModuleNotFoundError: No module named 'torchvision'

In [36]:
import detectron2

AttributeError: module 'torch' has no attribute '__version__'

In [37]:
from detectron2.utils.logger import setup_logger

In [38]:
import numpy as np
import cv2
import random
import glob
import torch
import time
import pickle
import os

from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import detection_utils as utils
from detectron2.data import samplers

from torch.utils.data import Dataset, DataLoader

AttributeError: module 'torch' has no attribute '__version__'

In [39]:
class PredictDataset(Dataset):
    
    def __init__(self, glob_string):
        self.image_files = sorted(glob.glob(glob_string))
        
    def __len__(self):
        return len(self.image_files) // 2
    
    def __getitem__(self, idx):
        image_raw = cv2.imread(self.image_files[idx * 2])
        height, width = image_raw.shape[:2]
        image = torch.as_tensor(image_raw.astype("float32").transpose(2, 0, 1)).contiguous()
        image_dict0 = {"image": image, "height": height, "width": width, "file_name": self.image_files[idx * 2]}
        
        image_raw = cv2.imread(self.image_files[idx * 2 + 1])
        height, width = image_raw.shape[:2]
        image = torch.as_tensor(image_raw.astype("float32").transpose(2, 0, 1)).contiguous()
        image_dict1 = {"image": image, "height": height, "width": width, "file_name": self.image_files[idx * 2 + 1]}
        return [image_dict0, image_dict1]

NameError: name 'Dataset' is not defined

In [ ]:
def cv2_imshow(im):
    cv2.imshow('file', im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
## writing all frames from a given thermal recording, separated into minute bins. Timestamps unrounded

zoom = 5

start_frame = 0

vid_length = 1*60*60

drive = 'E:/'

In [ ]:
date = '2019-08-14'

In [40]:
root = 'C:/Users/meerkat/Documents/thermal_baboon/'

In [41]:
write_to = root + 'DATA/images_to_process/' + date

In [42]:
## root on my local computer
# root = 'C:/Users/cloftus/Documents/thermal_baboons/annotations/'

## root on my HiWi computer
root = 'C:/Users/meerkat/Documents/thermal_baboon/'

## root on the server
#root = 'Y:/cloftus/analysis/thermal_baboon_project/'

from detectron2.data.datasets import register_coco_instances

output_folder = root + 'DATA/short_temp/all_coco_annotations'
coco_annotation_file = os.path.join(output_folder, 'baboon_coco_annotations.json')
output_image_folder = os.path.join(output_folder, 'images')

register_coco_instances("train",  {}, coco_annotation_file, output_image_folder ) 

train_metadata = MetadataCatalog.get("train")

AttributeError: module 'torch' has no attribute '__version__'

In [43]:
##base root on my local computer
#base_root = 'C:/Users/cloftus/'

##base root on the HiWi computer
base_root = 'C:/Users/meerkat/'

os.chdir( base_root + 'detectron2-windows/configs/COCO-Detection/' )

os.listdir()

['faster_rcnn_R_101_C4_3x.yaml',
 'faster_rcnn_R_101_DC5_3x.yaml',
 'faster_rcnn_R_101_FPN_3x.yaml',
 'faster_rcnn_R_50_C4_1x.yaml',
 'faster_rcnn_R_50_C4_3x.yaml',
 'faster_rcnn_R_50_DC5_1x.yaml',
 'faster_rcnn_R_50_DC5_3x.yaml',
 'faster_rcnn_R_50_FPN_1x.yaml',
 'faster_rcnn_R_50_FPN_3x.yaml',
 'faster_rcnn_X_101_32x8d_FPN_3x.yaml',
 'fast_rcnn_R_50_FPN_1x.yaml',
 'output',
 'retinanet_R_101_FPN_3x.yaml',
 'retinanet_R_50_FPN_1x.yaml',
 'retinanet_R_50_FPN_3x.yaml',
 'rpn_R_50_C4_1x.yaml',
 'rpn_R_50_FPN_1x.yaml']

In [44]:
cfg = get_cfg()
cfg.merge_from_file(
    base_root + 'detectron2-windows/configs/COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml'
)

baboon_weights = root + "RESULTS/detectron_output/output/full-aug_maxiter-2000_lr-0.019_detectPerIm-200_minsize-0_batchsize-8/model_final.pth"



cfg.MODEL.WEIGHTS = (baboon_weights)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.4
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (100)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 3
cfg.TEST.DETECTIONS_PER_IMAGE = 100

NameError: name 'get_cfg' is not defined

In [45]:
os.chdir( drive + date.replace( '-', '') )

In [46]:
thermal_file = glob.glob('*.seq')

In [47]:
thermal_file

['20190814_17_00_00-226_14_00_04_981.seq']

In [48]:
im = fnv.file.ImagerFile( thermal_file[0] )

In [49]:
im.get_frame(0)

True

In [50]:
im

In [ ]:

im.get_frame(0)

print( 'got_frame')

day1 = int(im.frame_info[0]['value'][:3])

start = date + ' ' + im.frame_info[0]['value'][4:]

start_dt = dt.datetime.strptime(start, '%Y-%m-%d %H:%M:%S.%f')

## save the first timestamp. We will add one to the date from this timestamp for all times that occur after midnight
og_start_dt = start_dt

## make the directory where we will write the images to
os.makedirs ( write_to + '/', exist_ok = True)

os.chdir( write_to + '/')

## set up the plot areas
final = np.array(im.final, copy=False).reshape((im.height, im.width))
    
width = final.shape[1] / (100 / zoom)
    
height = final.shape[0] / (100 / zoom)

fig = plt.figure( figsize = ( width, height ) )  

print( 'got_frame')


## I am not exactly sure what this if else statement does, but it has something to do with starting on a new video that is not starting at the beginning timestamp in case we are not starting at the first frame
if start_frame > 0:
    
    jumper = 0
    
else:
    
    jumper = 1
    
for i in range(start_frame, im.num_frames ): ## for each frame
    
    ## this will stop the code if there is an error with reading in the frame (which happens when reading from the server)
    if im.get_frame( i ) == False:
        blaasdfadff
        
    print( 'made it' )
    ## read in the frame
    im.get_frame(i)

    print( 'got_frame')

    ## pull the day of the frame
    day2 = int(im.frame_info[0]['value'][:3])

    ## determine if this is the same day as when the video started (i.e. is it past midnight?)
    day_diff = day2 - day1

    ## save the timestamp of the frame
    timestamp = f'{og_start_dt.year:04}-' + f'{og_start_dt.month:02}-' + f'{(og_start_dt.day+day_diff):02} ' + im.frame_info[0]['value'][4:]

    ## turn the character string timestamp into an actual datetime timestamp
    timestamp = dt.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
    
    ## if it has been an hour since the start of the first frame of this video, pause the frame printing, and make a video from the frames of the previous hour
    if timestamp > ( start_dt + dt.timedelta( seconds = vid_length ) ):
        
        print( 'heyo' )
        ## if we started at the beginning of the video, make a video of the frames in the past hour. If we didn't start at the beginning of the video, set the start_dt to the current timestamp, and we will start the new video at this frame (if we are on the first frame -- not the first frame of the video, but the first frame that we input above)
        if not jumper == 0:
           
            ## prepare the name of the video. This is the timestamp of the first frame of the video
            str_name = start_dt.strftime("%Y%m%d_%H%M%S%f")

            
            ### and now doing prediction within the frames in the past hour
            # FOLDER THAT HAS THE IMAGES YOU WANT TO PROCESS
            images_folder = write_to

            dataset = PredictDataset(os.path.join(images_folder, "*.png"))

            save_root = root + 'RESULTS/processed_images/' + date
            
            os.makedirs(save_root, exist_ok = True)

            data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers = 0 )
            
            cuda0 = torch.device('cuda:0')
            
            
            max_batches = 10000

            t = time.time()
            with torch.no_grad():
                for batch_num, image_batch in enumerate(data_loader):
                    if batch_num >= max_batches:
                        break

                    if batch_num % 250 == 0:
                        print('{} images processed'.format(batch_num * 2))
                    for i in range(len(image_batch)):
                        image_batch[i]['image'] = np.squeeze(image_batch[i]['image'])

                        image_batch[i]['image'] = image_batch[i]['image'].to(cuda0)
                        image_batch[i]['width'] = image_batch[i]['width'].to(cuda0).item()
                        image_batch[i]['height'] = image_batch[i]['height'].to(cuda0).item()
                        #print(image_batch['image'].shape)
                        #print(image_batch)
                    predictions = model(image_batch)
                    for preds, im_dict in zip(predictions, image_batch):
                        name = os.path.splitext(os.path.basename(im_dict['file_name'][0]))[0]
                        file = os.path.join(save_root, '{}-predictions.pkl'.format(name))
                        preds_instance = preds["instances"].to("cpu")
                        with open(file, 'wb') as out:
                            pickle.dump(preds_instance, out)
                            out.close()

            print(time.time() - t, flush = True )
            np_detections_file = os.path.join(save_root, '{}_detections.npy'.format(name))

            files = sorted(glob.glob(os.path.join(save_root, '*-predictions.pkl')))

            all_detections = []
            raw_instances = []

            for file in files[:]:
                with open(file, 'rb') as readfile:
                    detections=pickle.load(readfile)
                    #print( detections )
                detection_dict = detections.get_fields()
                detection_dict['pred_boxes'] = detection_dict['pred_boxes'].tensor.numpy()
                detection_dict['scores'] = detection_dict['scores'].numpy()
                detection_dict['pred_classes'] = detection_dict['pred_classes'].numpy()
                detection_dict['image_name'] = os.path.basename(file).split('-')[0]
                all_detections.append(detection_dict)
                raw_instances.append(detections)

            np_detections_file = os.path.join(save_root, '{}_detections.npy'.format(name) )
            np.save(np_detections_file, all_detections)
            
            os.chdir( save_root )
            
            counter = 0

            for detection in all_detections:

                print(detection['scores'].shape)
                print(detection['image_name'])

                img = plt.imread( root + 'DATA/images_to_process/' + detection['image_name'] + '.tiff' )

                #img = cv2.cvtColor( img, cv2.COLOR_BGR2RGB )
                plt.imshow( img )

                # Get the current reference
                ax = plt.gca()

                ax.axes.xaxis.set_visible(False)
                ax.axes.yaxis.set_visible(False)

                for item in detection['pred_boxes']:

                    x1 = item[ 0 ]

                    x2 = item[ 2 ]

                    y1 = item[ 1 ]

                    y2 = item[ 3 ]

                    wid = x2 - x1

                    hei = y2 - y1

                    # Create a Rectangle patch
                    rect = patches.Rectangle( (x1, y1 ), wid, hei, linewidth=1, edgecolor='c', facecolor='none' )

                    # Add the patch to the Axes
                    ax.add_patch( rect )

                plt.savefig( save_root + '/' + detection['image_name'] , bbox_inches = 'tight' )

                fig.clear()

                counter += 1

            asdfgafaafd
            
            name = image_batch[0]['file_name'][0].split('/')[-2]
            
            # make a text_file of all the tiffs in the folder where we have been printing the frames
            text_file = open(str_name + '.txt', 'w')

            files = sorted( os.listdir() )

            for file in files:

                if 'png' in file:

                    n = text_file.write('file ' + "'" + file + "'" + '\n')

            text_file.close()

            ## command line script for turning the frames we printed and saved into a video
            txt_name = str_name + '.txt'
            vid_name = str_name + '.mp4'

            command = 'ffmpeg -f concat -r 60 -i ' + pipes.quote( txt_name ) + ' -c:v libx264 -crf 15 -pix_fmt yuv420p -vf pad=ceil(iw/2)*2:ceil(ih/2)*2 ' + pipes.quote(vid_name)
            check_call(shlex.split(command))

        
        to_delete = glob.glob('*.png')
        
        for item in to_delete:
            os.remove( item )
        
        start_dt = timestamp
        
        jumper = 1
        #os.makedirs ( write_to + '/' + start_dt.strftime("%Y%m%d_%H%M%S%f") )
        
        #os.chdir( write_to + '/' + start_dt.strftime("%Y%m%d_%H%M%S%f") )
     
    
    #print( 'if_statement: ' , time.time() - start__t )

    #start__ = time.time()
        
        
    print( 'ahoy there' )
    final = np.array(im.final, copy=False).reshape( ( im.height, im.width ) )

    final1 = final - final.mean()
    
    #print( 'matrix: ' , time.time() - start__ )

    #start__ = time.time()

    print( 'here??' )
    sns.heatmap(final1, vmin = -np.std(final1), vmax = final1.max(), xticklabels = False, yticklabels = False, cbar = False)

    print( 'did you make it here?')
    if final1.shape[1] > 1000:

        plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 55*zoom / 5 )

    else:

        plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 25*zoom / 5 )

    
    #print( 'plotting: ' , time.time() - start__ )

    #start__ = time.time()

    print( 'what about here?' )
    plt.savefig( f'{i:06}_' + timestamp.strftime("%Y%m%d_%H%M%S%f") + '.png' , bbox_inches = 'tight')
    
    #print( 'save_plot: ' , time.time() - start__ )

    #start__ = time.time()
    
    print( 'hooray' )
    
    fig.clear()
    
    #plt.clf()
    
    #plt.cla()

    #plt.close( fig )
    #plt.close( 'all' )
    
    del day2, day_diff, final, final1
    
    gc.collect()
    
    #print( 'clean_up: ' , time.time() - start__ )
    
    #print( 'total_iter: ' , time.time() - start__t )


    
    #h = hpy()
    #print(h.heap())


got_frame
got_frame
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hooray
made it
got_frame
ahoy there
here??
did you make it here?
what about here?
hoo

In [ ]:
os.getcwd()

In [ ]:
str_name = start_dt.strftime("%Y%m%d_%H%M%S%f")

# make a text_file of all the tiffs in the folder
text_file = open(str_name + '.txt', 'w')

files = sorted( os.listdir() )

for file in files:

    
    if 'png' in file:

        n = text_file.write('file ' + "'" + file + "'" + '\n')

text_file.close()

## command line script for turning the series of pictures produced in the script below to movies for upload

txt_name = str_name + '.txt'
vid_name = str_name + '.mp4'

command = 'ffmpeg -f concat -r 60 -i ' + pipes.quote( txt_name ) + ' -c:v libx264 -crf 15 -pix_fmt yuv420p -vf pad=ceil(iw/2)*2:ceil(ih/2)*2 ' + pipes.quote(vid_name)
check_call(shlex.split(command))


#to_delete = glob.glob('*.png')

#for item in to_delete:
#    os.remove( item )

In [ ]:
to_delete = glob.glob('*.png')

for item in to_delete:
    os.remove( item )